In [1]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

In [2]:
#!/usr/bin/env python

__author__ = "Jing Zhang"
__email__ = "jingzbu@gmail.com"
__status__ = "Development"


from util import *
import numpy as np
from numpy.linalg import inv, matrix_rank
import json

In [3]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix_ext.pkz')
P = np.matrix(P)

In [4]:
print(matrix_rank(P))

462


In [5]:
print(np.size(P,0), np.size(P,1))

(462, 462)


In [6]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix_ext.pkz')

In [7]:
# assert(1 == 2)

# load link counts data
with open('../temp_files/link_day_minute_Apr_dict_ext_JSON_insert_links_adjusted.json', 'r') as json_file:
    link_day_minute_Apr_dict_ext_JSON = json.load(json_file)

In [8]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 22, 18, 19, 20, 23, 24, 25, 26, 27, 30]
# week_day_Apr_list = [22, 18, 19, 20, 23]

In [9]:
link_day_minute_Apr_list = []
for link_idx in range(74):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_ext_JSON[key] ['PM_flow_minute'][minute_idx])

In [10]:
print(len(link_day_minute_Apr_list))

186480


In [11]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 74, 2520)
x[x < 1] = 200
# x = np.nan_to_num(x)
# y = np.array(np.transpose(x))
# y = y[np.all(y != 0, axis=1)]
# x = np.transpose(y)
# x = np.matrix(x)

In [12]:
# print(np.size(x,0), np.size(x,1))
# print(x[:,:2])
# print(np.size(A,0), np.size(A,1))

In [13]:
# load node-link incidence matrix
N = zload('../temp_files/node_link_incidence_MA_ext.pkz')

In [14]:
N.shape

(22, 74)

In [15]:
n = 22  # number of nodes
m = 74  # number of links

In [16]:
x_0 = [x[:,2][i, 0] for i in range(m)]

In [17]:
# x_0

In [18]:
OD_pair_label_dict = zload('../temp_files/OD_pair_label_dict_ext.pkz')

In [19]:
len(OD_pair_label_dict)

462

In [20]:
L = 22 * (22 - 1)  # dimension of lam

In [21]:
# od pair correspondence
OD_pair_label_dict_MA_small = zload('../temp_files/OD_pair_label_dict__MA.pkz')

In [22]:
# create a dictionary mapping nodes of small network to nodes of bigger network
nodeToNode = {}

nodeList = range(9)[1:]
nodeListExt = [1, 4, 7, 12, 13, 14, 16, 17]
for i in nodeList:
    nodeToNode[str(i)] = nodeListExt[i-1]
# nodeToNode['1'] = 1
# nodeToNode['2']

In [23]:
nodeToNode

{'1': 1, '2': 4, '3': 7, '4': 12, '5': 13, '6': 14, '7': 16, '8': 17}

In [24]:
odMap = {}
for i in range(len(OD_pair_label_dict_MA_small)):
    key = str(i)
    origiSmall = OD_pair_label_dict_MA_small[key][0]
    destiSmall = OD_pair_label_dict_MA_small[key][1]
    origiExt = nodeToNode[str(origiSmall)]
    destiExt = nodeToNode[str(destiSmall)]
    odMap[key] = (origiExt, destiExt)

In [25]:
odMap

{'0': (1, 4),
 '1': (1, 7),
 '10': (4, 13),
 '11': (4, 14),
 '12': (4, 16),
 '13': (4, 17),
 '14': (7, 1),
 '15': (7, 4),
 '16': (7, 12),
 '17': (7, 13),
 '18': (7, 14),
 '19': (7, 16),
 '2': (1, 12),
 '20': (7, 17),
 '21': (12, 1),
 '22': (12, 4),
 '23': (12, 7),
 '24': (12, 13),
 '25': (12, 14),
 '26': (12, 16),
 '27': (12, 17),
 '28': (13, 1),
 '29': (13, 4),
 '3': (1, 13),
 '30': (13, 7),
 '31': (13, 12),
 '32': (13, 14),
 '33': (13, 16),
 '34': (13, 17),
 '35': (14, 1),
 '36': (14, 4),
 '37': (14, 7),
 '38': (14, 12),
 '39': (14, 13),
 '4': (1, 14),
 '40': (14, 16),
 '41': (14, 17),
 '42': (16, 1),
 '43': (16, 4),
 '44': (16, 7),
 '45': (16, 12),
 '46': (16, 13),
 '47': (16, 14),
 '48': (16, 17),
 '49': (17, 1),
 '5': (1, 16),
 '50': (17, 4),
 '51': (17, 7),
 '52': (17, 12),
 '53': (17, 13),
 '54': (17, 14),
 '55': (17, 16),
 '6': (1, 17),
 '7': (4, 1),
 '8': (4, 7),
 '9': (4, 12)}

In [26]:
odIdxExt = []  # OD pair idx in the extended network corresponding to the OD pairs in smaller network

for i in range(len(odMap)):
    odIdxExt.append(OD_pair_label_dict[str(odMap[str(i)])])

In [27]:
with open('../temp_files/OD_demand_matrix_Apr_weekday_PM.json', 'r') as json_file:
    demandsSmall = json.load(json_file)

In [28]:
# demandsSmall

In [29]:
# implement GLS method to estimate OD demand matrix
def GLS_Apr(x, A, P, L):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    A: path-link incidence matrix
    P: logit route choice probability matrix
    L: dimension of lam
    ----------------
    return: lam
    ----------------
    """
    K = np.size(x, 1)
    S = samp_cov(x)

    #print("rank of S is: \n")
    #print(matrix_rank(S))
    #print("sizes of S are: \n")
    #print(np.size(S, 0))
    #print(np.size(S, 1))

    inv_S = inv(S).real

    A_t = np.transpose(A)
    P_t = np.transpose(P)
    # PA'
    PA_t = np.dot(P, A_t)

    #print("rank of PA_t is: \n")
    #print(matrix_rank(PA_t))
    #print("sizes of PA_t are: \n")
    #print(np.size(PA_t, 0))
    #print(np.size(PA_t, 1))

    # AP_t
    AP_t = np.transpose(PA_t)

    Q_ = np.dot(np.dot(PA_t, inv_S), AP_t)
    Q = adj_PSD(Q_).real  # Ensure Q to be PSD
#     Q = Q_

    #print("rank of Q is: \n")
    #print(matrix_rank(Q))
    #print("sizes of Q are: \n")
    #print(np.size(Q, 0))
    #print(np.size(Q, 1))

    b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])
    # print(b[0])
    # assert(1==2)

    model = Model("OD_matrix_estimation")

    lam = []
    for l in range(L):
        lam.append(model.addVar(name='lam_' + str(l)))

    model.update() 

    # Set objective: (K/2) lam' * Q * lam - b' * lam
    obj = 0
    for i in range(L):
        for j in range(L):
            obj += (1.0 / 2) * K * lam[i] * Q[i, j] * lam[j]
    for l in range(L):
        obj += - b[l] * lam[l]
        
    model.setObjective(obj)

    # Add constraint: lam >= 0
    for l in range(L):
        model.addConstr(lam[l] >= 0)
        #model.addConstr(lam[l] <= 5000)
    fictitious_OD_list = zload('../temp_files/fictitious_OD_list')
    for l in fictitious_OD_list:
        model.addConstr(lam[l] == 0)
        
    for j in range(len(odMap)):
        model.addConstr(lam[odIdxExt[j]] - demandsSmall[str(j)] <= 0.2 * demandsSmall[str(j)])
        model.addConstr(demandsSmall[str(j)] - lam[odIdxExt[j]] <= 0.2 * demandsSmall[str(j)])
        
    model.update() 

    model.setParam('OutputFlag', False)
    model.optimize()

    lam_list = []
    for v in model.getVars():
        # print('%s %g' % (v.varName, v.x))
        lam_list.append(v.x)
    # print('Obj: %g' % obj.getValue())
    return lam_list

lam_list = GLS_Apr(x, A, P, L)

# write estimation result to file
n = 22  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_PM_ext.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1